In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = 'https://www.genie.co.kr/chart/top200'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}
req = requests.get(url, headers=header)

In [ ]:
#1,2,3,4 페이지
'''
https://www.genie.co.kr/chart/top200?ditc=D&ymd=20220808&hh=10&rtm=Y&pg=1
https://www.genie.co.kr/chart/top200?ditc=D&ymd=20220808&hh=10&rtm=Y&pg=2
https://www.genie.co.kr/chart/top200?ditc=D&ymd=20220808&hh=10&rtm=Y&pg=3
https://www.genie.co.kr/chart/top200?ditc=D&ymd=20220808&hh=10&rtm=Y&pg=4
'''

In [8]:
base_url = 'https://www.genie.co.kr/chart/top200?ditc=D&ymd=20220808&hh=10&rtm=Y&pg='

url = base_url + str(1)
req = requests.get(url, headers=header)

soup = BeautifulSoup(req.text, 'html.parser')

###1. 원하는 목록 찾기

In [9]:
trs = soup.select('tr.list') #tr 태그의 클래스명이 list
#body-content > div.newest-list > div > table > tbody > tr:nth-child(1)
len(trs)

50

### 2. 하나의 데이터를 선택해서 원하는 정보 추출하기

In [10]:
tr = trs[0]

In [15]:
rank = tr.select_one('.number').get_text()
rank

'1\n                                        \n                                    \n유지\n\n'

In [16]:
rank = rank.split('\n')[0]

In [17]:
title = tr.select_one('.title.ellipsis').get_text().strip()
title

'그때 그 순간 그대로 (그그그)'

In [19]:
artist = tr.select_one('.artist.ellipsis').get_text().strip()
artist

'WSG워너비 (가야G)'

In [20]:
album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
album

'WSG워너비 1집'

In [24]:
for tr in trs:
  title = tr.select_one('.title.ellipsis').get_text().strip()
  artist = tr.select_one('.artist.ellipsis').get_text().strip()
  album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
  rank = int(tr.select_one('.number').get_text().split('\n')[0])


4. 모든 페이지에 대해서 가져오기

In [30]:
lines = []
for page in range(1,5):
  url = base_url + str(page)
  req = requests.get(url, headers=header)
  soup = BeautifulSoup(req.text, 'html.parser')
  trs = soup.select('tr.list')
  for tr in trs:
    rank = int(tr.select_one('.number').get_text().split('\n')[0])
    title = tr.select_one('.title.ellipsis').string
    artist = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
    lines.append([title, artist, album, rank])

df = pd.DataFrame(lines, columns=['rank','artist', 'album', 'title'])


In [31]:
df.head(95).tail()

,rank,artist,album,title
90,\n \n ...,김호중,PANORAMA,91
91,None,GAYLE,abcdefu,92
92,\n \n ...,김호중,PANORAMA,93
93,\n \n ...,Peder Elias,Love & Loneliness,94
94,\n \n ...,BE'O (비오),Bipolar,95


### 6. 4과정을 버그를 잡을때까지 수정

In [45]:
lines = []
for page in range(1,5):
  url = base_url + str(page)
  req = requests.get(url, headers=header)
  soup = BeautifulSoup(req.text, 'html.parser')
  trs = soup.select('tr.list')
  for index,tr in enumerate(trs):
    try:
      title = tr.select_one('.title.ellipsis').string.strip()
    except:
      title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
    rank = int(tr.select_one('.number').get_text().split('\n')[0])
    artist = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
    lines.append([title, artist, album, rank])

In [37]:
tr = trs[27]
tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()

'Peaches (Feat. Daniel Caesar & Giveon)'

In [48]:
df = pd.DataFrame(lines, columns=['title','artist', 'album', 'rank'])

df.head(95).tail()

,title,artist,album,rank
90,친구,김호중,PANORAMA,91
91,abcdefu,GAYLE,abcdefu,92
92,Oye Como Va,김호중,PANORAMA,93
93,Loving You Girl (Feat. Hkeem),Peder Elias,Love & Loneliness,94
94,문득,BE'O (비오),Bipolar,95


In [50]:
df.to_csv('지니Top200(220808).csv',index=False)